<a href="https://colab.research.google.com/github/nisha235/pyspark/blob/main/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=01ffcd0a0a3aa1152cfd9ad313eafcc7d3824d6daff07054576375883f3e5ca7
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


#QUES 1
#Get the individual key from nested dictionary



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode

spark = SparkSession.builder.appName("ExpandNestedData").getOrCreate()

# Sample data as a list of tuples
data = [(10, [{'k1':20,'k2':40},{'k1':60, 'k2':80}]),(20, [{'k1':30,'k2':50},{'k1':90, 'k2':90}]) ]
df = spark.createDataFrame(data, ["col1", "nested_data"])
df.show()

+----+--------------------+
|col1|         nested_data|
+----+--------------------+
|  10|[{k1 -> 20, k2 ->...|
|  20|[{k1 -> 30, k2 ->...|
+----+--------------------+



In [ ]:

# Explode the nested list
exploded_df = df.withColumn("data", explode("nested_data"))
exploded_df.show()
# Select and rename columns
poutput = exploded_df.select("col1", "data.k1", "data.k2")
poutput.show()

+----+--------------------+--------------------+
|col1|         nested_data|                data|
+----+--------------------+--------------------+
|  10|[{k1 -> 20, k2 ->...|{k1 -> 20, k2 -> 40}|
|  10|[{k1 -> 20, k2 ->...|{k1 -> 60, k2 -> 80}|
|  20|[{k1 -> 30, k2 ->...|{k1 -> 30, k2 -> 50}|
|  20|[{k1 -> 30, k2 ->...|{k1 -> 90, k2 -> 90}|
+----+--------------------+--------------------+

+----+---+---+
|col1| k1| k2|
+----+---+---+
|  10| 20| 40|
|  10| 60| 80|
|  20| 30| 50|
|  20| 90| 90|
+----+---+---+



#QUES 2
Exploding list of list

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, array_max, col

spark = SparkSession.builder.appName("ExplodeNestedArray").getOrCreate()

# Sample data as a list of tuples
data = [([[[1,2,3], [2,3,4],[3,4,5,6]]])]
from pyspark.sql.functions import expr

# Create a DataFrame from the data
df = spark.createDataFrame(data, ["nested_data"])
df.show()
# Explode each nested list
exploded_df = df.withColumn("data", explode("nested_data"))
exploded_df.show()

exploded_df = exploded_df.withColumn('max', array_max(col('data')))
row1 = exploded_df.agg({"max": "max"}).collect()[0]
print(row1)
# Select and rename columns
exploded_df.select([expr('data[' + str(x) + ']') for x in range(0, 4)]).show()




+--------------------+
|         nested_data|
+--------------------+
|[[1, 2, 3], [2, 3...|
+--------------------+

+--------------------+------------+
|         nested_data|        data|
+--------------------+------------+
|[[1, 2, 3], [2, 3...|   [1, 2, 3]|
|[[1, 2, 3], [2, 3...|   [2, 3, 4]|
|[[1, 2, 3], [2, 3...|[3, 4, 5, 6]|
+--------------------+------------+

Row(max(max)=6)
+-------+-------+-------+-------+
|data[0]|data[1]|data[2]|data[3]|
+-------+-------+-------+-------+
|      1|      2|      3|   NULL|
|      2|      3|      4|   NULL|
|      3|      4|      5|      6|
+-------+-------+-------+-------+



#QUES 3
Get the frequency of unique values in the entire dataframe df.


In [4]:
#QUES 3
#Get the frequency of unique values in the entire dataframe df.


from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode

spark = SparkSession.builder.appName("frequency").getOrCreate()

data = [(1, 2, 3),
(2, 3, 4),
(1, 2, 3),
(4, 5, 6),
(2, 3, 4)]
df = spark.createDataFrame(data, ["Column1", "Column2", "Column3"])


# Print DataFrame
df.show()
# get column names
columns = df.columns


# stack all columns into a single column
df_single = None


for c in columns:
  if df_single is None:
    df_single = df.select(col(c).alias("single_column"))
  else:
    df_single = df_single.union(df.select(col(c).alias("single_column")))


# generate frequency table
frequency_table = df_single.groupBy("single_column").count().orderBy('count', ascending=False)


# show frequency table
frequency_table.show()






+-------+-------+-------+
|Column1|Column2|Column3|
+-------+-------+-------+
|      1|      2|      3|
|      2|      3|      4|
|      1|      2|      3|
|      4|      5|      6|
|      2|      3|      4|
+-------+-------+-------+

+-------------+-----+
|single_column|count|
+-------------+-----+
|            2|    4|
|            3|    4|
|            4|    3|
|            1|    2|
|            5|    1|
|            6|    1|
+-------------+-----+



#QUE 4
Appplication of REGEX_EXTRACT and REPPLACE, EXPPR AND SELECT

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract, col, expr, split, element_at


# Create a SparkSession
spark = SparkSession.builder \
   .appName("RegExp and Replace Example") \
   .getOrCreate()


# Sample DataFrame
data = [("mkt_id: 736482|user-click|landing_page",),
       ("mkt_id: 839274|user-view|homepage",),
       ("mkt_id: 128374|user-click|product_page",)]


df = spark.createDataFrame(data, ["mkt_code"])
df.show(truncate=False)


# Apply transformations
transformed_df = df.withColumn("product_marketing_id",
                              expr("REPLACE(REGEXP_EXTRACT(mkt_code, 'mkt_id:[^|]*', 0), 'mkt_id:', '')"))

# Extract the user-click value using regexp_extract


#transformed_df = transformed_df.withColumn("product_view",regexp_extract(col("mkt_code"), '\|(.*?)\|', 1))
#transformed_df = transformed_df.withColumn("product_page",regexp_extract(col("mkt_code"), '\|[-_a-z]+\|(.*?)$', 1))
# Show transformed DataFrame
#transformed_df.show(truncate=False)
split_df = transformed_df.withColumn("split_array", split("mkt_code", "\|"))

# Extract the landing_page value using element_at (last element)
extracted_df = split_df.withColumn("product_view", element_at("split_array", 2))
extracted_df = extracted_df.withColumn("product_page", element_at("split_array", -1))  # -1 for last element

# Show transformed DataFrame
extracted_df.show(truncate=False)


+--------------------------------------+
|mkt_code                              |
+--------------------------------------+
|mkt_id: 736482|user-click|landing_page|
|mkt_id: 839274|user-view|homepage     |
|mkt_id: 128374|user-click|product_page|
+--------------------------------------+

+--------------------------------------+--------------------+------------------------------------------+------------+------------+
|mkt_code                              |product_marketing_id|split_array                               |product_view|product_page|
+--------------------------------------+--------------------+------------------------------------------+------------+------------+
|mkt_id: 736482|user-click|landing_page| 736482             |[mkt_id: 736482, user-click, landing_page]|user-click  |landing_page|
|mkt_id: 839274|user-view|homepage     | 839274             |[mkt_id: 839274, user-view, homepage]     |user-view   |homepage    |
|mkt_id: 128374|user-click|product_page| 128374          

#Que 5
To find the start and end flight from the dataset

In [31]:
from pyspark.sql import Row
from pyspark.sql import Window
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

# Create a Spark session

spark = SparkSession.builder.appName("CreateDataFrameExample").getOrCreate()

data = [ ('c1', 'New York', 'Lima'),
   ('c1', 'London', 'New York'),
   ('c1', 'Lima', 'Sao Paulo'),
   ('c1', 'Sao Paulo', 'New Delhi'),
   ('c2', 'Mumbai', 'Hyderabad'),
   ('c2', 'Surat', 'Pune'),
   ('c2', 'Hyderabad', 'Surat'),
   ('c3', 'Kochi', 'Kurnool'),
   ('c3', 'Lucknow', 'Agra'),
   ('c3', 'Agra', 'Jaipur'),
   ('c3', 'Jaipur', 'Kochi')]

schema = "customer string , start_location string , end_location string"
df = spark.createDataFrame(data = data , schema = schema)
df.show()

df_start = df.select(col("customer"), col("start_location").alias("loc"))
df_end = df.select(col("customer"), col("end_location").alias("loc"))

df_union = df_start.union(df_end)

df_unique = df_union.groupBy(col("customer") , col("loc")).agg(
   count(lit(1)).alias("cn")
).filter(col("cn") ==1).drop(col("cn")).orderBy(col("customer"))

df_unique.show()
df_answer = df.join(df_unique, on= ((df.customer == df_unique.customer) &(df.start_location == df_unique.loc) | (df.end_location == df_unique.loc)), how='inner').drop(df.customer)
df_answer = df_answer.withColumn('cust_start_loc', when(col('start_location')==col('loc'), col('start_location'))).withColumn('cust_end_loc', when(col('end_location')==col('loc'), col('end_location'))).select(col("customer"), col("cust_start_loc"), col("cust_end_loc"))
df_answer.show()
df_answer = df_answer.groupby(col('customer')).agg(min(col('cust_start_loc')).alias('start_loation'),min(col('cust_end_loc')).alias('end_loation'))
df_answer.show()

+--------+--------------+------------+
|customer|start_location|end_location|
+--------+--------------+------------+
|      c1|      New York|        Lima|
|      c1|        London|    New York|
|      c1|          Lima|   Sao Paulo|
|      c1|     Sao Paulo|   New Delhi|
|      c2|        Mumbai|   Hyderabad|
|      c2|         Surat|        Pune|
|      c2|     Hyderabad|       Surat|
|      c3|         Kochi|     Kurnool|
|      c3|       Lucknow|        Agra|
|      c3|          Agra|      Jaipur|
|      c3|        Jaipur|       Kochi|
+--------+--------------+------------+

+--------+---------+
|customer|      loc|
+--------+---------+
|      c1|   London|
|      c1|New Delhi|
|      c2|   Mumbai|
|      c2|     Pune|
|      c3|  Lucknow|
|      c3|  Kurnool|
+--------+---------+

+--------+--------------+------------+
|customer|cust_start_loc|cust_end_loc|
+--------+--------------+------------+
|      c1|        London|        NULL|
|      c1|          NULL|   New Delhi|
|      c